In [1]:
import pandas as pd
import numpy as np

data = pd.read_pickle("TCR_epitope_embedding_data.pkl")

df_emb = data[['epi','tcr','epi_embeds','tcr_embeds','binding']]#.sample(frac=1,random_state=1).reset_index()
print("Training Data loaded\n")

Training Data loaded



In [4]:
counts = df_emb['epi'].value_counts().index.tolist()
d = df_emb.loc[(df_emb['epi']==counts[0])]
for i in range(1,6):
    d = pd.concat([d,df_emb.loc[(df_emb['epi']==counts[i])]])
d = d.reset_index()
data = d[['epi','tcr','epi_embeds','tcr_embeds','binding']]
data['epi'].value_counts().index.tolist()

['KLGGALQAK',
 'MIELSLIDFYLCFLAFLLFLVLIML',
 'YVLDHLIVV',
 'GLCTLVAML',
 'HTTDPSFLGRY',
 'GILGFVFTL']

In [5]:
data = data.sample(frac=1).reset_index(drop=True)
data

,epi,tcr,epi_embeds,tcr_embeds,binding
0,MIELSLIDFYLCFLAFLLFLVLIML,CASSQGPTPTSYEQYF,"[0.6477096676826477, -0.9077708721160889, -0.6...","[-0.16367115080356598, -0.13441194593906403, -...",1
1,MIELSLIDFYLCFLAFLLFLVLIML,CASSRLAGTEQFF,"[0.6041505336761475, -0.8648455739021301, -0.5...","[0.05388971418142319, -0.24742507934570312, -0...",0
2,GILGFVFTL,CASSRNANYGYTF,"[0.749771773815155, -0.44504648447036743, -0.2...","[0.22062858939170837, 0.04196928068995476, 0.2...",0
3,MIELSLIDFYLCFLAFLLFLVLIML,CASSPIGTGGGGYTF,"[0.6143473982810974, -0.9444854259490967, -0.6...","[0.02350204810500145, -0.16862230002880096, 0....",0
4,KLGGALQAK,CASSLRQAYNEQFF,"[0.9116198420524597, 0.08816317468881607, -0.5...","[-0.1849283128976822, -0.16282175481319427, -0...",1
...,...,...,...,...,...
103067,KLGGALQAK,CASSQDWGGETQYF,"[0.9831998348236084, 0.21545323729515076, -0.5...","[-0.034669745713472366, -0.3386443555355072, -...",1
103068,MIELSLIDFYLCFLAFLLFLVLIML,CAGQALQATNQPQHF,"[0.605818510055542, -0.9020426869392395, -0.61...","[0.11295557022094727, -0.34101802110671997, -0...",0
103069,KLGGALQAK,CATVSLDTEAFF,"[0.9116199016571045, 0.08816313743591309, -0.5...","[-0.22858275473117828, -0.46227848529815674, -...",0
103070,KLGGALQAK,CSASGDREVSGEKLFF,"[0.911614716053009, 0.08814360201358795, -0.50...","[0.3348677158355713, -0.11847081035375595, -0....",1


In [6]:
N = len(data)

trainData = data[:int(N*5/6)]
testData = data[int(N*5/6):]

trainData['epi'].value_counts().index.tolist()

['KLGGALQAK',
 'MIELSLIDFYLCFLAFLLFLVLIML',
 'YVLDHLIVV',
 'GLCTLVAML',
 'HTTDPSFLGRY',
 'GILGFVFTL']

In [7]:
testData['epi'].value_counts().index.tolist()

['KLGGALQAK',
 'MIELSLIDFYLCFLAFLLFLVLIML',
 'YVLDHLIVV',
 'GLCTLVAML',
 'HTTDPSFLGRY',
 'GILGFVFTL']

In [9]:
trainData.to_pickle("NewData/trainData_epi_5.pkl")
testData.to_pickle("NewData/testData_epi_5.pkl")


In [4]:
import sys
!{sys.executable} -m pip install -U scikit-learn

import numpy as np      
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support,roc_auc_score, precision_score, recall_score, f1_score, accuracy_score

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
def load_data_split(dat,split_type, seed):
    n_fold = 5
    idx_test_fold = 0
    idx_val_fold = -1
    idx_test = None
    idx_train = None
    x_pep = dat.epi
    x_tcr = dat.tcr
    
    if split_type == 'random':
        n_total = len(x_pep)
    elif split_type == 'epi':
        unique_peptides = np.unique(x_pep)
        n_total = len(unique_peptides)
    elif split_type == 'tcr':
        unique_tcrs = np.unique(x_tcr)
        n_total = len(unique_tcrs)
        
    np.random.seed(seed)    
    idx_shuffled = np.arange(n_total)
    np.random.shuffle(idx_shuffled)
    
    # Determine data split from folds
    n_test = int(round(n_total / n_fold))
    n_train = n_total - n_test

    # Determine position of current test fold
    test_fold_start_index = idx_test_fold * n_test
    test_fold_end_index = (idx_test_fold + 1) * n_test

    if split_type == 'random':
        # Split data evenly among evenly spaced folds
        # Determine if there is an outer testing fold
        if idx_val_fold < 0:
            idx_test = idx_shuffled[test_fold_start_index:test_fold_end_index]
            idx_train = list(set(idx_shuffled).difference(set(idx_test)))
        else:
            validation_fold_start_index = args.idx_val_fold * n_test
            validation_fold_end_index = (args.idx_val_fold + 1) * n_test
            idx_test_remove = idx_shuffled[test_fold_start_index:test_fold_end_index]
            idx_test = idx_shuffled[validation_fold_start_index:validation_fold_end_index]
            idx_train = list(set(idx_shuffled).difference(set(idx_test)).difference(set(idx_test_remove)))
    elif split_type == 'epi':
        if idx_val_fold < 0:
            idx_test_pep = idx_shuffled[test_fold_start_index:test_fold_end_index]
            test_peptides = unique_peptides[idx_test_pep]
            idx_test = [index for index, pep in enumerate(x_pep) if pep in test_peptides]
            idx_train = list(set(range(len(x_pep))).difference(set(idx_test)))
        else:
            validation_fold_start_index = args.idx_val_fold * n_test
            validation_fold_end_index = (args.idx_val_fold + 1) * n_test
            idx_test_remove_pep = idx_shuffled[test_fold_start_index:test_fold_end_index]
            test_remove_peptides = unique_peptides[idx_test_remove_pep]
            idx_test_pep = idx_shuffled[validation_fold_start_index:validation_fold_end_index]
            test_peptides = unique_peptides[idx_test_pep]
            idx_test = [index for index, pep in enumerate(x_pep) if pep in test_peptides]
            idx_test_remove = [index for index, pep in enumerate(x_pep) if pep in test_remove_peptides]
            idx_train = list(set(range(len(x_pep))).difference(set(idx_test)).difference(set(idx_test_remove)))
    elif split_type == 'tcr':
        if idx_val_fold < 0:
            idx_test_tcr = idx_shuffled[test_fold_start_index:test_fold_end_index]
            test_tcrs = unique_tcrs[idx_test_tcr]
            idx_test = [index for index, tcr in enumerate(x_tcr) if tcr in test_tcrs]
            idx_train = list(set(range(len(x_tcr))).difference(set(idx_test)))
        else:
            validation_fold_start_index = args.idx_val_fold * n_test
            validation_fold_end_index = (args.idx_val_fold + 1) * n_test
            idx_test_remove_tcr = idx_shuffled[test_fold_start_index:test_fold_end_index]
            test_remove_tcrs = unique_tcrs[idx_test_remove_tcr]
            idx_test_tcr = idx_shuffled[validation_fold_start_index:validation_fold_end_index]
            test_tcrs = unique_tcrs[idx_test_tcr]
            idx_test = [index for index, tcr in enumerate(x_tcr) if tcr in test_tcrs]
            idx_test_remove = [index for index, tcr in enumerate(x_tcr) if tcr in test_remove_tcrs]
            idx_train = list(set(range(len(x_tcr))).difference(set(idx_test)).difference(set(idx_test_remove)))

    testData = dat.iloc[idx_test, :].sample(frac=1).reset_index(drop=True)
    trainData = dat.iloc[idx_train, :].sample(frac=1).reset_index(drop=True)
    

    print('================check Overlapping========================')
    print('number of overlapping tcrs: ', str(len(set(trainData.tcr).intersection(set(testData.tcr)))))
    print('number of overlapping epitopes: ', str(len(set(trainData.epi).intersection(set(testData.epi)))))
    
    return trainData, testData

In [6]:
trainData, testData = load_data_split(data,'epi', 21)
len(trainData), len(testData)

================check Overlapping========================
number of overlapping tcrs:  196
number of overlapping epitopes:  0


(75248, 17022)

In [8]:
trainData['epi'].value_counts().index.tolist()

['KLGGALQAK', 'MIELSLIDFYLCFLAFLLFLVLIML', 'GLCTLVAML', 'HTTDPSFLGRY']

In [9]:
testData['epi'].value_counts().index.tolist()

['YVLDHLIVV']

In [ ]:
trainData.to_pickle('NewData/trainData_5epi_split.pkl')
testData.to_pickle('NewData/testData_5epi_split.pkl')
pd.read_pickle('NewData/trainData_5epi.pkl')

,epi,tcr,epi_embeds,tcr_embeds,binding
0,YVLDHLIVV,CASSEQANEQYF,"[0.7756434679031372, -0.9900071024894714, -0.6...","[-0.1736331582069397, -0.43690457940101624, -0...",1
1,GLCTLVAML,CSARDRTGDGHTF,"[0.7244112491607666, -0.7280503511428833, -0.4...","[0.07484224438667297, -0.33770039677619934, -0...",1
2,KLGGALQAK,CSAETSGSSYEQYF,"[0.9116251468658447, 0.08816703408956528, -0.5...","[0.08830253034830093, 0.08487285673618317, -0....",0
3,GLCTLVAML,CSARDGTGNGFTF,"[0.7209442853927612, -0.7276771068572998, -0.4...","[0.11971393972635269, -0.45145565271377563, -0...",1
4,KLGGALQAK,CASSSGTSAYPGRQETQYF,"[0.9113970994949341, 0.08795130252838135, -0.5...","[0.2172468602657318, 0.3224843740463257, -0.27...",0
...,...,...,...,...,...
94999,KLGGALQAK,CASGLVPENTEAFF,"[0.9116476774215698, 0.08816434442996979, -0.5...","[0.023342767730355263, -0.3201124370098114, -0...",1
95000,GLCTLVAML,CASTKRKAKNIQYF,"[0.7203195095062256, -0.6220163702964783, -0.5...","[-0.09611504524946213, 0.016997234895825386, 0...",0
95001,KLGGALQAK,CSATRITSGRAGGELFF,"[0.9117439985275269, 0.08874913305044174, -0.5...","[0.025452837347984314, 0.07424432039260864, -0...",1
95002,KLGGALQAK,CASSLRDQSSSEQYF,"[0.9139075875282288, 0.0904158502817154, -0.51...","[-0.17554189264774323, -0.13312700390815735, -...",0


In [ ]:
import pandas as pd
pd.read_pickle('NewData/trainData_5epi.pkl')

,epi,tcr,epi_embeds,tcr_embeds,binding
0,YVLDHLIVV,CASSEQANEQYF,"[0.7756434679031372, -0.9900071024894714, -0.6...","[-0.1736331582069397, -0.43690457940101624, -0...",1
1,GLCTLVAML,CSARDRTGDGHTF,"[0.7244112491607666, -0.7280503511428833, -0.4...","[0.07484224438667297, -0.33770039677619934, -0...",1
2,KLGGALQAK,CSAETSGSSYEQYF,"[0.9116251468658447, 0.08816703408956528, -0.5...","[0.08830253034830093, 0.08487285673618317, -0....",0
3,GLCTLVAML,CSARDGTGNGFTF,"[0.7209442853927612, -0.7276771068572998, -0.4...","[0.11971393972635269, -0.45145565271377563, -0...",1
4,KLGGALQAK,CASSSGTSAYPGRQETQYF,"[0.9113970994949341, 0.08795130252838135, -0.5...","[0.2172468602657318, 0.3224843740463257, -0.27...",0
...,...,...,...,...,...
94999,KLGGALQAK,CASGLVPENTEAFF,"[0.9116476774215698, 0.08816434442996979, -0.5...","[0.023342767730355263, -0.3201124370098114, -0...",1
95000,GLCTLVAML,CASTKRKAKNIQYF,"[0.7203195095062256, -0.6220163702964783, -0.5...","[-0.09611504524946213, 0.016997234895825386, 0...",0
95001,KLGGALQAK,CSATRITSGRAGGELFF,"[0.9117439985275269, 0.08874913305044174, -0.5...","[0.025452837347984314, 0.07424432039260864, -0...",1
95002,KLGGALQAK,CASSLRDQSSSEQYF,"[0.9139075875282288, 0.0904158502817154, -0.51...","[-0.17554189264774323, -0.13312700390815735, -...",0


In [ ]:
pd.read_pickle('NewData/testData_5epi.pkl')

,epi,tcr,epi_embeds,tcr_embeds,binding
0,MIELSLIDFYLCFLAFLLFLVLIML,CASSPGLAGGSTDTQYF,"[0.6147972345352173, -0.8775625824928284, -0.6...","[-0.01643841713666916, -0.007422896102070808, ...",1
1,MIELSLIDFYLCFLAFLLFLVLIML,CASSVSGTGLNYGYTF,"[0.6186230182647705, -0.8805468678474426, -0.6...","[-0.008796021342277527, -0.037387244403362274,...",0
2,MIELSLIDFYLCFLAFLLFLVLIML,CATSEGDENYEQYF,"[0.629840075969696, -0.9370735883712769, -0.63...","[-0.28216952085494995, -0.18787583708763123, -...",0
3,MIELSLIDFYLCFLAFLLFLVLIML,CASRKNSATNEKLFF,"[0.6121242642402649, -0.8972762823104858, -0.5...","[0.23919309675693512, -0.13959814608097076, -0...",1
4,MIELSLIDFYLCFLAFLLFLVLIML,CASGGTRYNEQFF,"[0.6333950161933899, -0.8769041299819946, -0.6...","[-0.1817336082458496, -0.1438559591770172, -0....",1
...,...,...,...,...,...
23141,MIELSLIDFYLCFLAFLLFLVLIML,CASSLFKGAQSAQHF,"[0.6007687449455261, -0.8907893896102905, -0.6...","[0.12155120819807053, -0.3903921842575073, -0....",0
23142,MIELSLIDFYLCFLAFLLFLVLIML,CSASLGLAYEQYF,"[0.5787399411201477, -0.8995267748832703, -0.5...","[-0.19039307534694672, -0.25985607504844666, -...",1
23143,MIELSLIDFYLCFLAFLLFLVLIML,CASSYSTRGARRTQYF,"[0.5553742051124573, -0.9407874941825867, -0.5...","[-0.10607048869132996, -0.3162108063697815, 0....",0
23144,MIELSLIDFYLCFLAFLLFLVLIML,CASSFKIGGESNQPQHF,"[0.6140031218528748, -0.8955808877944946, -0.6...","[-0.04727771505713463, -0.1714484840631485, -0...",0


: 